# Feature Hashing

大多数机器学习算法, 都需要输入的数据是实数矩阵. 而从原始数据到实数矩阵的过程就是特征工程的工作. Feature Hashing ( aka. Hash Trick ) 是特征工程的一种技术.

通常处理类别型特征的 One-Hot-Encoding 相比, Hash Trick 有以下优点和存在的问题:

优点: 
- 实现非常简单
- 不需要提前准备映射的字典, ( OHE 需要提前将所有类别型特征的值和二进制编码一一对应起来, 然后这个映射关系, 在训练和应用阶段都需要使用, 在有超多值的特征的情况下, 这就是一笔很大的开销 )
- 适用于在线学习, ( 因为 OHE 没办法提前对所有数据准备映射 )

问题:
- 哈希碰撞 (实践得出, 通常碰撞不怎么影响模型的精度)
- 经过 特征Hash 之后的特征的值没有可解释性, 也不可逆


WIKI 上简单实现的伪代码如下, N 为输出特性向量的维度

```
function hashing_vectorizer(features : array of string, N : integer):
     x := new vector[N]
     for f in features:
         h := hash(f)
         x[h mod N] += 1
     return x
```

在实际使用中, 我们把所有原始特征的特征 Name 和 Value 进行拼接后作为输入 (e.g. category_1101); 而输出的值在预先设定好的 [0,N] 范围内, 直接可以转化为 N 维的稀疏向量. 

至于 N 的选择是需要调整的; 过小的维度会导致过多的碰撞, 而过大的维度, 又会导致模型参数的维度过大 (通常选择为 2^10 左右).

## Ref

- https://stackoverflow.com/questions/8673035/what-is-feature-hashing-hashing-trick
- https://www.quora.com/Can-you-explain-feature-hashing-in-an-easily-understandable-way?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

## Appendix

In [9]:
import pandas as pd
import numpy as np

data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}

data = pd.DataFrame(data)

def hash_col(df, col, N):
    cols = [col + "_" + str(i) for i in range(N)]
    def xform(x): tmp = [0 for i in range(N)]; tmp[hash(x) % N] = 1; return pd.Series(tmp,index=cols)
    df[cols] = df[col].apply(xform)
    return df.drop(col,axis=1)
print(data)
hash_col(data, 'state', 4)

   pop   state  year
0  1.5    Ohio  2000
1  1.7    Ohio  2001
2  3.6    Ohio  2002
3  2.4  Nevada  2001
4  2.9  Nevada  2002


,pop,year,state_0,state_1,state_2,state_3
0,1.5,2000,0,0,1,0
1,1.7,2001,0,0,1,0
2,3.6,2002,0,0,1,0
3,2.4,2001,0,1,0,0
4,2.9,2002,0,1,0,0
